#Criar um script que faça:  
#Pegue as nfe em xml do email  
#Extrai os dados necessarios   
#Tratar dados   
#Remover materiais pesado E outras notas 
-TABUAS CABOQUINHO  
-CIMENTRO VOTORAN  
-Magalu   

#Aplicador formula de taxas  
#Inserir no template dos marketplaces
  

In [1]:
import os
import shutil
import xml.etree.ElementTree as ET
import pandas as pd
from imap_tools import MailBox, AND
from config import pwd, user
from datetime import date
import openpyxl
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
#Baixa os anexos XML dos emails
save_folder = "notas/nfes/"
os.makedirs(save_folder, exist_ok=True)

with MailBox("imap.gmail.com").login(user, pwd, initial_folder="INBOX", ) as mailbox:
    list_mail = mailbox.fetch(criteria=AND(date=date.today())

    )
    for email in list_mail:
        for anexo in email.attachments:
            if  anexo.filename.lower().endswith(".xml"):
                file_path = os.path.join(save_folder, anexo.filename)
                if not os.path.exists(file_path):   
                    with open(file_path, "wb") as f:
                        f.write(anexo.payload)

In [18]:
#Salva os XMLs em pastas por Emitente e extrai os produtos para um Excel

pasta_origem = "notas/nfes"
todos_produtos = []


def extrai_dados(caminho_arquivo):
    produtos = []
    tree = ET.parse(caminho_arquivo)
    root = tree.getroot()
    ns = {"ns": "http://www.portalfiscal.inf.br/nfe"}
    
    natOp = root.find(".//ns:ide/ns:natOp", ns)
    if natOp is not None and natOp.text.strip().upper() == 'BONIFICACAO, DOACAO OU BRINDE':
        return [] 
    
    for det in root.findall(".//ns:det", ns):   
        produto = {}
        produto ['Codigo Produto'] = det.find("./ns:prod/ns:cProd", ns).text
        produto ['Descrição'] = det.find("./ns:prod/ns:xProd", ns).text
        produto ['Valor_unitário'] = det.find("./ns:prod/ns:vUnCom", ns).text
        produto ['Código de Barras'] = det.find("./ns:prod/ns:cEAN", ns).text 
        produto ['Sku'] = produto['Código de Barras']
        if produto['Sku'] == 'SEM GTIN':
            produto['Sku'] = produto['Descrição']
        produto ['Fornecedor'] = root.find(".//ns:emit/ns:xNome", ns).text
        produtos.append(produto)
    return produtos
        


# Percorre todos os arquivos da pasta
for arquivo in os.listdir(pasta_origem):
    if arquivo.lower().endswith(".xml"):
        caminho_arquivo = os.path.join(pasta_origem, arquivo)

        try:
            # Abre e parseia o XML
            tree = ET.parse(caminho_arquivo)
            root = tree.getroot()

            # Define namespace
            ns = {"ns": "http://www.portalfiscal.inf.br/nfe"}

            # Busca o CNPJ do emitente
            cnpj_emitente = root.find(".//ns:emit/ns:CNPJ", ns)
            cnpj_emitente
            if cnpj_emitente is None:
                print(f"⚠️ CNPJ não encontrado em {arquivo}")
                continue

            cnpj_emitente = cnpj_emitente.text

            produtos = extrai_dados(caminho_arquivo)
            todos_produtos.extend(produtos)              


            emitente = root.find(".//ns:emit/ns:xNome", ns)
            pasta_destino = os.path.join("notas/", emitente.text)

            # Cria a pasta se não existir
            os.makedirs(pasta_destino, exist_ok=True)

            # Copia o arquivo para a pasta
            shutil.copy(caminho_arquivo, pasta_destino)

        except Exception as e:
            print(f" {arquivo}: {e}")


df = pd.DataFrame(todos_produtos)


final = df.drop_duplicates(subset=['Sku'], keep='first')


fornecedores_pesados = [
    'IND E COM DE TUBOS E CONEXOES FORT.COM',
    'VOTORANTIM CIMENTOS SA',
    'CABOQUINHO MATERIAIS PARA CONSTRUCAO'
]

final = final[~final['Fornecedor'].isin(fornecedores_pesados)]


final.to_excel("produtos_extraidos.xlsx", index=False)
sku2 = df.query('Sku == Descrição')
sem_gtin = sku2.drop_duplicates(subset=['Codigo Produto'], keep='first')
sem_gtin.to_excel("sem_gtin.xlsx", index=False)

In [ ]:
sem_gtin = pd.read_excel('sem_gtin.xlsx')

options = Options()
options.add_argument("--headless")  # modo sem interface gráfica
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

for index, produtos_sem_gtin in sem_gtin.iterrows():
    fornecedor = produtos_sem_gtin['Fornecedor']
    codigo_produto = produtos_sem_gtin['Codigo Produto']
    descricao = produtos_sem_gtin['Descrição']


    if fornecedor == 'CONSTRUDIGI DISTRIBUIDORA DE MATERIAIS PARA CONSTRUCAO LTDA':
        url = f'https://www.construdigi.com.br/produto/{codigo_produto}/{descricao}'
    elif fornecedor == 'M.S.B. COMERCIO DE MATERIAIS PARA CONSTRUCAO':
        url = f'https://msbitaqua.com.br/produto/{codigo_produto}/{descricao}'
    elif fornecedor == "CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU":
        url = f'https://www.construja.com.br/produto/{codigo_produto}/{descricao}'
    else:
        continue
        

    driver.get(url)

    try:
    # espera até 10s pelo elemento "Cód. Barras"
        gtin_valor = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//td[text()='Cód. Barras']/following-sibling::td"))
    ).text.strip()


        sem_gtin.at[index, 'Código de Barras'] = gtin_valor
        sem_gtin.at[index, 'Sku'] = gtin_valor

    except:
        print(f"Cód. Barras não encontrado{descricao}")



driver.quit()


sem_gtin.to_excel("sem_gtin_atualizado.xlsx", index=False)


In [56]:
arq1 = pd.read_excel('produtos_extraidos.xlsx')
arq2 = pd.read_excel('sem_gtin_atualizado.xlsx')

produtos_atualizados = pd.concat([arq1,arq2], ignore_index=True)

produto = produtos_atualizados.drop_duplicates(subset=['Codigo Produto'], keep='first')
produto.to_excel("produtos_final.xlsx", index=False)


In [2]:
produto = pd.read_excel('produtos_final.xlsx')

chunck_size = 100

for i in range(0, len(produto), chunck_size):
    chunk = produto[i:i + chunck_size]
    chunk.to_excel(f'produtos_chunk_{i//chunck_size + 1}.xlsx', index=False)


In [ ]:


options = Options()
options.add_argument("--headless")  # modo sem interface gráfica
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

for xl in os.listdir('chunks/'):
    chunk = pd.read_excel(xl)

    for index, produtos in chunk.iterrows():
        fornecedor = produtos['Fornecedor']
        codigo_produto = produtos['Codigo Produto']
        descricao = produtos['Descrição']


        if fornecedor == 'CONSTRUDIGI DISTRIBUIDORA DE MATERIAIS PARA CONSTRUCAO LTDA':
            url = f'https://www.construdigi.com.br/produto/{codigo_produto}/{descricao}'
        elif fornecedor == 'M.S.B. COMERCIO DE MATERIAIS PARA CONSTRUCAO':
            url = f'https://msbitaqua.com.br/produto/{codigo_produto}/{descricao}'
        elif fornecedor == "CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU":
            url = f'https://www.construja.com.br/produto/{codigo_produto}/{descricao}'
        else:
            continue
            
        driver.get(url) 

        try:
            peso_valor = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//td[text()='Peso Bruto']/following-sibling::td"))
            ).text.strip()


            produto.at[index, 'Peso Bruto'] = peso_valor
            print("Código encontrado:", peso_valor, descricao)

        except:
            print(f"Peso não encontrado não encontrado{descricao}")



    driver.quit()


    produto.to_excel("produtos_final.xlsx", index=False)

